### [tf.while_loop](https://www.tensorflow.org/api_docs/python/tf/while_loop)

In [8]:
import tensorflow as tf
import numpy as np

In [3]:
with tf.Session() as sess:
    i = tf.constant(0)
    condition = lambda i: tf.less(i, 10)
    body = lambda i: tf.add(i, 1)
    r = tf.while_loop(condition, body, [i])
    print(sess.run(r))

10


In [7]:
import collections
Pair = collections.namedtuple('Pair', 'j, k')
with tf.Session() as sess:
    ijk_0 = (tf.constant(0), Pair(tf.constant(1), tf.constant(2)))
    c = lambda i, p: i < 10
    b = lambda i, p: (i + 1, Pair((p.j + p.k), (p.j - p.k)))
    ijk_final = tf.while_loop(c, b, ijk_0)
    print(sess.run(ijk_final))


(10, Pair(j=32, k=64))


In [9]:
def body(x):
    a = tf.random_uniform(shape=[2, 2], dtype=tf.int32, maxval=100)
    b = tf.constant(np.array([[1, 2], [3, 4]]), dtype=tf.int32)
    c = a + b
    return tf.nn.relu(x + c)

def condition(x):
    return tf.reduce_sum(x) < 100

x = tf.Variable(tf.constant(0, shape=[2, 2]))

with tf.Session():
    tf.initialize_all_variables().run()
    result = tf.while_loop(condition, body, [x])
    print(result.eval())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[20 82]
 [94 90]]


In [18]:
elems = np.array([[1,2], [2,6], [3,6], [4,6], [5,6], [6,6]])
elems.shape

(6, 2)

In [24]:
squares = tf.map_fn(lambda x: x * x, elems)

with tf.Session() as sess:
    print(sess.run(squares))
print(tf.get_default_session())    
print(squares.eval(session=tf.get_default_session()))

[[ 1  4]
 [ 4 36]
 [ 9 36]
 [16 36]
 [25 36]
 [36 36]]
None


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [10]:
import numpy as np
import tensorflow as tf

/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [92]:
a = np.random.randint(-1, 5, (3,4,2))
a

array([[[ 2,  3],
        [ 4,  2],
        [-1,  0],
        [ 4,  4]],

       [[ 2, -1],
        [ 2, -1],
        [ 2,  3],
        [ 3,  3]],

       [[-1,  2],
        [ 3,  0],
        [ 4,  2],
        [ 2,  3]]])

In [95]:

def get_sequence_length(sequence):
    '''
    Returns the sequence length, droping out all the zeros if the sequence is padded
    :param sequence: Tensor(shape=[batch_size, doc_length, feature_dim])
    :return: Array of Document lengths of size batch_size
    '''
    used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
    length = tf.reduce_sum(used,1)
    length = tf.cast(length, tf.int32)
    return length

In [100]:
3 * 4 * 2

24

In [94]:
#3 samples, doc_length=4+2+1, num_words = 2  ===> 3 * 7 * 2 = 42
#3 samples, doc_length=4, num_words = 2 PAdded ====> 3 * 4 * 2 = 24
a = np.array([[
        [ 2,  3],
        [ 4,  2],
        [ 1,  0],
        [ 4,  4]],

       [[ 2, 1],
        [ 2, 1],
        [ -2,  -3],
        [ -3,  -3]],

       [[1,  2],
        [ -3,  -2],
        [ -4,  -2],
        [ -2,  -3]]])
a

array([[[ 2,  3],
        [ 4,  2],
        [ 1,  0],
        [ 4,  4]],

       [[ 2,  1],
        [ 2,  1],
        [-2, -3],
        [-3, -3]],

       [[ 1,  2],
        [-3, -2],
        [-4, -2],
        [-2, -3]]])

In [105]:
with tf.Session() as sess:
    char_ids = tf.Variable(a, dtype=tf.int32)
    value = tf.constant(0, tf.int32)
    
    length = get_sequence_length(char_ids)
    flag = tf.greater_equal(char_ids, 0)
    res = tf.gather_nd(indices=tf.where(flag), params=char_ids)
    res = tf.reshape(res, shape=(3, 2, -1))
    tf.initialize_all_variables()
    print(sess.run(char_ids, feed_dict={char_ids: a}))
    print(sess.run(length, feed_dict={char_ids: a}))
    print(sess.run(res, feed_dict={char_ids: a}))

[[[ 2  3]
  [ 4  2]
  [ 1  0]
  [ 4  4]]

 [[ 2  1]
  [ 2  1]
  [-2 -3]
  [-3 -3]]

 [[ 1  2]
  [-3 -2]
  [-4 -2]
  [-2 -3]]]
[4 4 4]


InvalidArgumentError: Input to reshape is a tensor with 14 values, but the requested shape requires a multiple of 6
	 [[Node: Reshape_32 = Reshape[T=DT_INT32, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](GatherNd_43/_11, Reshape_32/shape)]]
	 [[Node: Reshape_32/_13 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_14_Reshape_32", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Reshape_32', defined at:
  File "/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/site-packages/ipykernel_launcher

In [28]:

x = tf.constant([1, 2, 0, 4])
y = tf.Variable([1, 2, 0, 4])
mask = a > 0
slice_y_greater_than_one = tf.boolean_mask(a, mask)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print (sess.run(slice_y_greater_than_one))

[3 1 3 1 2 3 2 1 2]
